# Camada Gold - OBT de Crédito

---

## Objetivo
Este é o artefato final do pipeline. A Camada Gold consolida as informações limpas e tipadas da Camada Silver em um **Produto de Dados** altamente otimizado (One Big Table - OBT). Esta tabela foi modelada especificamente para responder às dores de duas áreas de negócio o **Time de Modelagem (Risco/Data Science)** e o **Time de Políticas de Crédito**.

## Modelagem e Feature Engineering
1. **Prevenção de Explosão Cartesiana (Fan-out):** Antes de realizar os cruzamentos, as tabelas satélites de comportamento (Afastamentos e Observações) sofrem um `groupBy`. Múltiplos eventos no mesmo mês são condensados, garantindo o grão estrito de `1 CPF x 1 Posição Base` na tabela final, prevenindo duplicação de renda.
2. **Criação de Flags de Risco (Booleanas):** Conversão da presença de registros de afastamento em variáveis binárias (`flag_afastado_mes` e `flag_observacao_restritiva`), formato ideal para algoritmos de Machine Learning no cálculo de *Credit Scoring*.
3. **Business Naming Convention:** Renomeação completa do esquema (ex: `ORG_LOTACAO` -> `orgao_lotacao`), entregando colunas semânticas e prontas para visualização (BI) ou exploração (Python/Pandas).

## Decisões de Engenharia
* **The Big Join:** A tabela de Cadastro foi eleita como "espinha dorsal" (Left Table), garantindo que todo servidor público seja mapeado na base, mesmo que não tenha recebido salário naquele mês específico.
* **Data Lineage:** Atendimento direto a um dos requisitos principais do desafio. A origem do dado foi preservada nas colunas `fonte_dado_cadastro` e `fonte_dado_remuneracao`, permitindo auditoria *row-level* retroativa.
* **Performance:** Materialização em formato **Delta Lake**, mantendo a chave de partição `posicao_base` para consultas otimizadas de evolução histórica (Time Series).

---
**Origem:** Múltiplas tabelas da Camada Silver (`public_informations.silver_...`)

**Destino:** Produto de Dados OBT (`public_informations.gold_servidores_credito`)

In [0]:
from pyspark.sql import functions as F
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

SCHEMA_NAME = "public_informations"

def build_gold_layer():
    logger.info("Iniciando processamento da Camada Gold: Tabela de Crédito e Políticas")
    
    try:
        df_cad = spark.read.table(f"{SCHEMA_NAME}.silver_cadastro")
        df_rem = spark.read.table(f"{SCHEMA_NAME}.silver_remuneracao")
        df_afa = spark.read.table(f"{SCHEMA_NAME}.silver_afastamentos")
        df_obs = spark.read.table(f"{SCHEMA_NAME}.silver_observacoes")
        
        # Agregação de Risco (Evitando duplicação de linhas)
        # Se o servidor tirou 2 licenças no mesmo mês, o join duplicaria a renda dele. 
        # Agrupamos para criar apenas uma Flag de "Afastado no Mês".
        df_afa_flags = df_afa.groupBy("Id_SERVIDOR_PORTAL", "posicao_base").agg(
            F.lit(True).alias("flag_afastado_mes")
        )
        
        df_obs_flags = df_obs.groupBy("Id_SERVIDOR_PORTAL", "posicao_base").agg(
            F.lit(True).alias("flag_observacao_restritiva")
        )
        
        # O Grande Join (A Espinha Dorsal é o Cadastro)
        df_gold = df_cad.alias("cad").join(
            df_rem.alias("rem"),
            on=["Id_SERVIDOR_PORTAL", "posicao_base"],
            how="left"
        ).join(
            df_afa_flags.alias("afa"),
            on=["Id_SERVIDOR_PORTAL", "posicao_base"],
            how="left"
        ).join(
            df_obs_flags.alias("obs"),
            on=["Id_SERVIDOR_PORTAL", "posicao_base"],
            how="left"
        )
        
        # Seleção e Renomeação de Features para Negócio (Business Naming)
        # Preenchendo valores nulos nas flags de risco com False
        df_final = df_gold.select(
            # Chaves e Tempo
            "posicao_base",
            "Id_SERVIDOR_PORTAL",
            F.col("cad.CPF").alias("cpf_mascarado"),
            
            # Features para Políticas de Crédito (Ocupação)
            F.col("cad.SITUACAO_VINCULO").alias("situacao_vinculo"),
            F.col("cad.DESCRICAO_CARGO").alias("cargo"),
            F.col("cad.ORG_LOTACAO").alias("orgao_lotacao"),
            F.col("cad.UF_EXERCICIO").alias("uf_exercicio"),
            F.col("cad.JORNADA_DE_TRABALHO").alias("jornada_trabalho"),
            
            # Features para Modelagem de Risco (Renda)
            F.col("rem.REMUNERAÇÃO_BÁSICA_BRUTA_R$").alias("renda_bruta"),
            F.col("rem.REMUNERAÇÃO_APÓS_DEDUÇÕES_OBRIGATÓRIAS_R$").alias("renda_liquida_estimada"),
            F.col("rem.IRRF_R$").alias("imposto_retido_fonte"),
            F.col("rem.GRATIFICAÇÃO_NATALINA_R$").alias("gratificacao_natalina"),
            
            # Flags Comportamentais / Risco
            F.coalesce(F.col("afa.flag_afastado_mes"), F.lit(False)).alias("flag_afastado_mes"),
            F.coalesce(F.col("obs.flag_observacao_restritiva"), F.lit(False)).alias("flag_observacao_restritiva"),
            
            # Auditoria e Linhagem (Requisito do PM: "identificar a fonte")
            F.col("cad.arquivo_origem").alias("fonte_dado_cadastro"),
            F.col("rem.arquivo_origem").alias("fonte_dado_remuneracao"),
            F.current_timestamp().alias("timestamp_carga_gold")
        )
        
        # Materialização da Tabela Final
        gold_table_name = f"{SCHEMA_NAME}.gold_servidores_credito"
        
        (df_final.write
         .format("delta")
         .mode("overwrite")
         .partitionBy("posicao_base")
         .saveAsTable(gold_table_name))
        
        logger.info(f" -> ✅ SUCESSO! Produto de dados finalizado: {gold_table_name}")
        
    except Exception as e:
        logger.error(f" -> ❌ Erro na Camada Gold: {e}", exc_info=True)

if __name__ == "__main__":
    build_gold_layer()

2026-02-25 22:03:24,853 - Iniciando processamento da Camada Gold: Tabela de Crédito e Políticas
2026-02-25 22:03:33,234 -  -> ✅ SUCESSO! Produto de dados finalizado: public_informations.gold_servidores_credito
